# 1) Preparación previa

In [ ]:
En la 3ra y última notebook se prueba una herramienta no vista en el curso. Se separó para ser planteada como un anexo, debido a nuestra incertidumbre respecto de sus resultados.

#### Carga de librerías

In [ ]:
import numpy as np
import pandas as pd
import warnings
import itertools
import numpy as np
import random
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

# !pip install fbprophet
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
import itertools
from fbprophet.diagnostics import performance_metrics

warnings.filterwarnings("ignore")

#### Lectura del dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Agustin-Bulzomi/Projects/main/Programming/Digital%20House%20(Python)/Support%20Files/Final%20Project/coin_Bitcoin.csv', delimiter=',')
df.head()

#### Creación de un nuevo DataFrame con las columnas necesarias para trabajar con Prophet:

In [ ]:
df2 = pd.DataFrame()

In [ ]:
df2['ds'] = df['Date']
df2['y']= df['Close']

In [ ]:
# Se cambia el formato de las fechas:
df2['ds'] = pd.to_datetime(df2['ds'])
df2['ds'].dtype

#### División del dataset en Train y Test, usando rangos personalizados:

In [ ]:
end_date = '2021-01-27'
mask1 = (df2['ds'] <= end_date)
mask2 = (df2['ds'] > end_date)

In [ ]:
X_tr = df2.loc[mask1]
X_tst = df2.loc[mask2]
print("train shape",X_tr.shape)
print("test shape",X_tst.shape)

#### Ploteo de los dos datasets obtenidos:

In [ ]:
pd.plotting.register_matplotlib_converters()
f, ax = plt.subplots(figsize=(14,5))
X_tr.plot(kind='line', x='ds', y='y', color='blue', label='Train', ax=ax)
X_tst.plot(kind='line', x='ds', y='y', color='red', label='Test', ax=ax)
plt.title('Rango para Train y para Test')
plt.show()

#### Se define una función para el cálculo del MAPE:

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# 2) Aplicación del modelo Prophet

#### Buscamos la mejor combinación de hiperparámetros:

In [ ]:
# Este código tarda mucho tiempo

param_grid = {  
    'changepoint_prior_scale': [0.01, 0.1, 0.5],
    'seasonality_prior_scale': [1.0, 10.0, 20.0],
    'seasonality_mode' : ('additive', 'multiplicative'),
    'daily_seasonality' : [False, True]
}

# Genera todas las combinaciones de parámetros
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Usa cross validation para evaluar los parámetros
for params in all_params:
    m = Prophet(**params).fit(X_tr)  # Fitea el modelo con los parámetros obtenidos
    df_cv = cross_validation(m, initial='2500 days' ,period= '15 days',horizon='31 days')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Encuentra los mejores parámetros
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

#### Análisis de cuál tuvo mejor rendimiento:

In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)

#### Aplicación de los hiperparámetros sobre el modelo y fiteo

In [ ]:
final_model = Prophet(changepoint_prior_scale=0.01 ,
                     seasonality_prior_scale = 1.0 ,
                     seasonality_mode= 'multiplicative',
                     daily_seasonality=False)
     
final_model.fit(X_tr)

#### Visualización de un forecast de 31 días con el modelo fiteado en train:

In [ ]:
future = final_model.make_future_dataframe(periods=31, freq='D')
forecast = final_model.predict(future)

forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

#### Finalmente, en la variable X_tst_final se guarda el predict en test:

In [ ]:
X_tst_final= final_model.predict(X_tst)
X_tst_final[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

#### Ploteo de resultados:

In [ ]:
f, ax = plt.subplots(figsize=(14,5))
f.set_figheight(5)
f.set_figwidth(15)
X_tst.plot(kind='line',x='ds', y='y', color='red', label='Actual', ax=ax)
X_tst_final.plot(kind='line',x='ds',y='yhat', color='green',label='Forecast', ax=ax)
plt.title('Forecast vs Actual')
plt.show()

#### Aplicación de la función MAPE

In [ ]:
MAPE = mean_absolute_percentage_error(X_tst['y'],abs(X_tst_final['yhat']))
print('MAPE', MAPE)

Es importante a destacar aquí que no se encuentra el sentido del MAPE, considerando que el RMSE fue de tan solo 6997 (lo cual matemáticamente se corresponde con un MAPE aproximado de 15%, que fue el que se insertó en la notebook 2). De ser 60 de MAPE, el resultado es muy negativo, mientras que de ser 15 se podría considerar al Prophet como un buen modelo